In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src import eda, util
import ipywidgets as widgets

# ---- EDA Konfiguration ----
EDA_FD001 = True
EDA_FD002 = True
EDA_FD003 = True
EDA_FD004 = True

# ---- Caching-/Recompute-Konfiguration ----
PRECOMPUTE_ALL_PLOTS = False        # Alle Plots bei Notebook-Start berechnen. Widgets reagieren erst nach dem Precompute.
FORCE_RECOMPUTE_PLOTS = False       # Alle Plots beim Precompute immer neu berechnen, auch wenn sie schon im Cache sind.
FORCE_RECOMPUTE_TSNE_DBSCAN = False # TSNE/DBSCAN immer neu berechnen

if not PRECOMPUTE_ALL_PLOTS:        # Nur gültig, wenn Precompute aktiviert ist!
    FORCE_RECOMPUTE_PLOTS = False   
    FORCE_RECOMPUTE_TSNE_DBSCAN = False

<table style="width:100%; background-color: white; padding: 10px; border-radius: 6px; box-shadow: 0 0 5px rgba(0,0,0,0.2);">
  <tr>
    <td>
      <h1 style="margin-bottom: 0; color: black; font-size: clamp(1.4rem, 2.2vw, 2.2rem);">
        Predictive Maintenance: Vorhersage der Remaining Useful Life (RUL) von Triebwerken des NASA C-MAPSS-Datensatzes
      </h1>
    </td>
    <td align="right">
      <img src="images/OST_Logo_DE_RGB@2000ppi.png" alt="OST Logo" width="200">
    </td>
  </tr>
</table>

**Autor:** Rino Albertin  
**Datum:** 6. März 2025

---
## Inhaltsverzeichnis

1. Einleitung  
   1.1 Zielsetzung  
   1.2 Vorgehensweise  

2. Überblick über den Datensatz und technische Grundlagen  
   2.1 Allgemeine Datenstruktur  
   2.2 Operative Einstellungen und Sensorik  

3. Einzelanalyse  
   3.1 FD001  
   3.2 FD002  
   3.3 FD003  
   3.4 FD004  

4. Kombinierte Analyse aller Datensätze  
   4.1 Explorative Datenanalyse  
   4.2 Feature Engineering  
   4.3 Modellierung  
   4.4 Hyperparameter-Tuning  
   4.5 Evaluation  

5. Vergleich der Modellresultate  

6. Fazit und Ausblick

# Anhang

---
## 1. Einleitung

Predictive Maintenance ist in der heutigen Industrie von zentraler Bedeutung, um ungeplante Ausfälle zu vermeiden und Wartungskosten zu senken. Die Vorhersage der verbleibenden Nutzungsdauer (Remaining Useful Life, RUL) von Triebwerken ermoeglicht eine optimierte Planung von Wartungsarbeiten und verbessert die Betriebseffizienz.

Der NASA (C-MAPSS) Commercial Modular Aero-Propulsion System Simulation-Datensatz bietet simulierte Run-to-Failure-Zeitreihen von Triebwerken der NASA, die Zeit, Betriebsbedingungen, 3 operative Einstellungen und 21 Sensormessungen umfassen. Dies macht ihn zu einer idealen Grundlage, um moderne Machine-Learning-Methoden zur RUL-Vorhersage zu evaluieren und zu vergleichen.

## 1.1 Zielsetzung

**Ziel:**  
Entwicklung eines Machine-Learning-Modells, das auf Basis historischer Sensordaten die RUL von Triebwerken präzise vorhersagen kann. Dabei soll sowohl das Verhalten einzelner Szenarien als auch eine generalisierbare Lösung über alle vier Datensätze hinweg untersucht werden.


## 1.2 Vorgehensweise

Die Bearbeitung erfolgt in zwei aufeinander aufbauenden Phasen:

### Phase 1: Einzelanalysen pro Datensatz (FD001–FD004)
Jeder der vier C-MAPSS-Datensätze wird separat analysiert, um spezifische Charakteristika und Herausforderungen zu verstehen. Für jeden Datensatz wird dieselbe Pipeline angewendet:

1. **Explorative Datenanalyse (EDA):**
   - Visualisierung typischer Sensorverläufe.
   - Analyse der Datenstruktur und Lebensdauerverläufe.

2. **Feature Engineering:**
   - Selektion und Transformation relevanter Sensoren.
   - Umgang mit korrelierten oder konstanten Features.

3. **Modellierung:**
   - Auswahl geeigneter Machine-Learning-Modelle.
   - Aufbau einer Pipeline (Preprocessing, Training, Inferenz).

4. **Hyperparameter-Tuning:**
   - Optimierung mit GridSearch oder Optuna.

5. **Evaluation:**
   - Validierung.
   - Vergleich der Szenarien und Modelle.

### Phase 2: Kombinierte Analyse (Generalmodell)
In Phase 2 werden die vier Datensätze zusammengeführt, um ein Modell zu entwickeln, das über unterschiedliche Szenarien hinweg verallgemeinerbar ist. Die Analyse basiert auf der gleichen Pipeline wie in Phase 1, mit folgenden Ergänzungen:

- **Berücksichtigung des Szenario-Kontexts:** Die Information, aus welchem C-MAPSS-Datensatz ein Triebwerk stammt, wird als zusätzliches Feature einbezogen (z. B. über One-Hot-Encoding).
- **EDA über alle Datensätze hinweg:** Gemeinsame Analyse von Verteilungen, Sensorwerten und Zykluslängen über alle Szenarien.
- **Evaluation der Generalisierungsfähigkeit:** Untersuchung, wie gut das kombinierte Modell mit der Heterogenität der Daten umgehen kann.

**Referenzen:**  
- NASA C-MAPSS-Datensatz: https://data.nasa.gov/d/ff5v-kuh6, Alternativ verfügbar unter: [Kaggle – NASA Turbofan Jet Engine Data Set](https://www.kaggle.com/datasets/behrad3d/nasa-cmaps)
- Machine Learning Unterichtsunterlagen der OST – Ostschweizer Fachhochschule
---

## 2. Überblick über den Datensatz und technische Grundlagen

Bevor mit der eigentlichen Analyse begonnen wird, erfolgt in diesem Kapitel eine technische Einführung in den C-MAPSS-Datensatz. Ziel ist es, ein grundlegendes Verständnis für Struktur, Sensorik und die inhaltlichen Rahmenbedingungen zu schaffen. Eine erste übergreifende EDA bietet zusätzlich einen Einblick in Gemeinsamkeiten und Besonderheiten der vier Szenarien.

Hinweis: Eine detaillierte, datensatzspezifische Analyse folgt in Kapitel 3–6, während Kapitel 7 die kombinierte Betrachtung behandelt.

### 2.1 Allgemeine Datenstruktur

Der C-MAPSS-Datensatz (Commercial Modular Aero-Propulsion System Simulation) besteht aus vier Teildatensätzen (FD001–FD004), die unterschiedliche Betriebsszenarien simulieren. Jeder Datensatz enthält Sensormessungen von mehreren Triebwerken über deren gesamte Lebensdauer hinweg (Run-to-Failure).

Für jede Zeile sind folgende Informationen verfügbar:

- **unit**: ID des Triebwerks (eine Einheit)
- **time**: aktueller Zyklus (Zeitpunkt)
- **op_setting_1–3**: operative Einstellungen (z. B. Flughöhe, Machzahl, TRA)
- **sensor_1–21**: Sensormessungen aus verschiedenen Triebwerkskomponenten

![Triebwerksaufbau in C-MAPSS](images/Simplified%20diagram%20of%20engine%20simulated%20in%20C-MAPSS.png)

Die Abbildung zeigt den schematischen Aufbau eines Triebwerks, wie es in der C-MAPSS-Simulation verwendet wird. Die Hauptkomponenten sind:

- **Fan:** Der Fan (Luftgebläse) saugt Umgebungsluft an, die teilweise in den Bypass strömt (äusserer Luftstrom) und teilweise ins Triebwerk.
- **LPC (Low Pressure Compressor):** Komprimiert die angesaugte Luft bei niedrigem Druck.
- **HPC (High Pressure Compressor):** Erhöht den Druck der Luft weiter vor der Verbrennung.
- **Combustor:** Vermischt die verdichtete Luft mit Treibstoff, zündet die Mischung und erzeugt damit heisse Hochdruckgase.
- **HPT (High Pressure Turbine):** Entzieht den heissen Gasen Energie, um den HPC anzutreiben.
- **LPT (Low Pressure Turbine):** Treibt den Fan und die LPC an.
- **N1/N2:** Repräsentieren die beiden Hauptwellen im Triebwerk (N1: Fan + LPC, N2: HPC + HPT).
- **Nozzle:** Düse am Austritt – beschleunigt die Abgasströmung und erzeugt Schub.

Viele der 21 Sensoren messen Parameter an genau diesen Stellen, z. B. Druck, Temperatur, Drehzahl oder Luftmassenströme.

### 2.2 Operative Einstellungen und Sensorik

Die C-MAPSS-Daten umfassen neben Zyklusinformationen auch drei operative Einstellungen und 21 Sensormessungen. Die operativen Einstellungen variieren je nach Szenario und beeinflussen die physikalischen Messgrössen, die an verschiedenen Triebwerkskomponenten aufgezeichnet werden.

#### Operative Einstellungen

| Spalte         | Beschreibung                         | Wertebereich        |
|----------------|--------------------------------------|---------------------|
| op_setting_1   | Flughöhe (Altitude)                  | 0 – 42'000 ft       |
| op_setting_2   | Machzahl                             | 0 – 0.84            |
| op_setting_3   | Throttle Resolver Angle (TRA)        | 20 – 100            |

#### Sensorübersicht

| Sensor | Beschreibung                             | Wertebereich (FD001–FD004) |
|--------|------------------------------------------|----------------------------|
|  1     | Total temperature at fan inlet (T2)      | 445.0 – 518.67 °R          |
|  2     | Total temperature at LPC outlet (T24)    | 535.48 – 645.11 °R         |
|  3     | Total temperature at HPC outlet (T30)    | 1242.67 – 1616.91 °R       |
|  4     | Total temperature at LPT outlet (T50)    | 1023.77 – 1441.49 °R       |
|  5     | Pressure at fan inlet (P2)               | 3.91 – 14.62 psia          |
|  6     | Bypass-duct pressure (P15)               | 5.67 – 21.61 psia          |
|  7     | Total pressure at HPC outlet (P30)       | 136.17 – 570.81 psia       |
|  8     | Physical fan speed (Nf)                  | 1914.72 – 2388.64 rpm      |
|  9     | Physical core speed (Nc)                 | 7984.51 – 9244.59 rpm      |
| 10     | Engine pressure ratio (epr)              | 0.93 – 1.32                |
| 11     | Static pressure at HPC outlet (Ps30)     | 36.04 – 48.53 psia         |
| 12     | Ratio of fuel flow to Ps30 (phi)         | 128.31 – 537.49 pps/psi    |
| 13     | Corrected fan speed (NRf)                | 2027.57 – 2390.49 rpm      |
| 14     | Corrected core speed (NRc)               | 7845.78 – 8293.72 rpm      |
| 15     | Bypass ratio (BPR)                       | 8.1563 – 11.0669           |
| 16     | Burner fuel-air ratio (farB)             | 0.02 – 0.03                |
| 17     | Bleed enthalpy (htBleed)                 | 302   – 400                |
| 18     | Demanded fan speed (Nf_dmd)              | 1915   – 2388 rpm          |
| 19     | Demanded corrected fan speed (PCNfR_dmd) | 84.93 – 100.0 rpm          |
| 20     | HPT coolant bleed (W31)                  | 10.16 – 39.89 lbm/s        |
| 21     | LPT coolant bleed (W32)                  | 6.0105 – 23.9505 lbm/s     |

*Die angegebenen Wertebereiche wurden über alle Trainingsdaten der Szenarien FD001 bis FD004 bestimmt.*

In [ ]:
# Load the CMAPSS dataset
df_train_01, df_test_01 = util.load_cmapss_data("FD001")
df_train_02, df_test_02 = util.load_cmapss_data("FD002")
df_train_03, df_test_03 = util.load_cmapss_data("FD003")
df_train_04, df_test_04 = util.load_cmapss_data("FD004")
df_train, df_test = util.load_cmapss_data()

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

---
## 3. Einzelanalysen der vier Datensätze

Ziel dieses Kapitels ist es, die vier Teildatensätze **FD001 bis FD004** separat zu analysieren und zu modellieren. Jeder Datensatz repräsentiert ein eigenes Betriebsszenario mit unterschiedlichen Rahmenbedingungen (z. B. konstante vs. variable Settings, unterschiedliche Komplexität der Sensorverläufe).

Für jeden Datensatz wird eine identische Analysepipeline angewendet, bestehend aus:
- Explorative Datenanalyse (EDA)
- Feature Engineering
- Modellierung
- Hyperparameter-Tuning
- Evaluation

Diese Einzelschritte ermöglichen ein besseres Verständnis der Stärken und Schwächen verschiedener Modelle je Szenario und legen die Basis für die spätere kombinierte Analyse in Kapitel 4.


***Hinweis zur EDA-Struktur:***  
Die explorative Datenanalyse (EDA) folgt in allen vier Szenarien derselben systematischen Gliederung:

1. Lebensdauerstatistiken
2. Operation Settings 
3. Sensorverhalten 
4. Sensorverteilungen nach Settings-Kategorien
5. Clusteranalyse

Jeder Abschnitt enthält gezielte Visualisierungen mit kurzer Interpretation. Ergänzend wird durch t-SNE und DBSCAN-Clusteranalyse die zugrunde liegende Zustandsstruktur der Triebwerke analysiert. Die so identifizierten Cluster bilden unterschiedliche Lebensphasen, Betriebsmodi und Degradationsverläufe ab und liefern die Grundlage für die Ableitung zustandsabhängiger Features im Feature Engineering.

Eine Übersicht und Erläuterung aller verwendeten Visualisierungstypen befindet sich in **Anhang A**. Die detaillierte Analyse aller beobachteten Muster, Korrelationen und Besonderheiten ist in **Anhang B** dokumentiert.

Ausreisser wurden nicht entfernt, da sie potenziell wichtige Hinweise auf kritische Phasen oder Zustandswechsel enthalten. Eine Glättung wurde bewusst vermieden, um das reale Verhalten der Triebwerke nicht zu verzerren.

---
### 3.1 Analyse für FD001
Der Datensatz FD001 stellt das einfachste Szenario innerhalb von C-MAPSS dar: konstante Betriebsbedingungen und ein einziger Degradationsmodus. Er eignet sich daher besonders gut für eine erste Modellierung und das Testen von Grundkonzepten.

#### 3.1.1 Explorative Datenanalyse (EDA)
Ziel dieses Abschnitts ist es, ein erstes Verständnis für die Struktur und Eigenschaften des Datensatzes FD001 zu entwickeln. Dabei werden typische Lebensdauerverläufe analysiert, exemplarische Sensorwerte visualisiert und erste Hinweise auf potenziell relevante Merkmale identifiziert.

In [ ]:
if EDA_FD001:
    # ─────────────────────────────
    # Konfiguration für FD001 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
        "overview": True,
        "ops": True,
        "sensors": True,
        "hue": True,
        "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True

    toggle = util.make_toggle_shortcut(df_train_01, "FD001")
    sensor_cols = [f"sensor_{i}" for i in [2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21]]
    df_copy = df_train_01.copy()
    df_copy["op_setting_1_cat"] = pd.qcut(df_copy["op_setting_1"], 3, labels=["tief", "mittel", "hoch"])
    df_copy["op_setting_2_cat"] = pd.qcut(df_copy["op_setting_2"], 3, labels=["tief", "mittel", "hoch"])

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]
    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[39, 69]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.25, upper_quantile=0.75),
        toggle("2-5. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
        toggle("2-6. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
    ]
    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window=10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=69, dataset_name="FD001-69"),
        toggle("3-3. Sensor-Korrelation", lambda df: (
            (fig := plt.figure(figsize=(28, 10))),
            (axs := fig.subplots(1, 2)),
            eda.plot_sensor_correlation_matrix(df, dataset_name="FD001", ax=axs[0]),
            eda.plot_sensor_correlation_matrix(df, sensor_cols=sensor_cols, dataset_name="FD001 (ohne konstante)", annot=True, ax=axs[1]),
            plt.tight_layout(),
        )),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle, sensor_cols=sensor_cols),
        toggle("3-5. Sensorverteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=sensor_cols),
        toggle("3-6. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation, sensor_cols=sensor_cols),
        toggle("3-7. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=sensor_cols),
    ]
    hue_plots = [
        toggle("4-1. Sensorverteilungen nach op_setting_1_cat",
               lambda df: eda.plot_sensor_distributions_by_cycle_range(
                   df=df_copy, lower_quantile=0.25, upper_quantile=0.75,
                   dataset_name="FD001", hue_col="op_setting_1_cat", sensor_cols=sensor_cols)),
        toggle("4-2. Sensorverteilungen nach op_setting_2_cat",
               lambda df: eda.plot_sensor_distributions_by_cycle_range(
                   df=df_copy, lower_quantile=0.25, upper_quantile=0.75,
                   dataset_name="FD001", hue_col="op_setting_2_cat", sensor_cols=sensor_cols)),
    ]

    # Einstellungen von plot_tsne_dbscan_clusters
    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_01,
            dataset_name="FD001",
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN
        )
        df_train_01["cluster_tsne"] = labels

    cluster_plots = [
        toggle("5-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters),  # plot_tsne_dbscan_clusters zeigt nur den Plot an
        toggle("5-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("5-3. Zustandsdauer je Cluster (Boxplot)", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("5-4. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("5-5. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD001"),
            fig.update_layout(width=1800, height=600),
            fig
        )),
        toggle("5-6. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("5-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("5-8. Trend Sensoren je Cluster",
               lambda df: util.make_cluster_navigation_panel(
                   df=df,
                   cluster_col="cluster_tsne",
                   cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                   sensor_cols=sensor_cols,
                   dataset_name="FD001",
                   force_recompute=not USE_CACHE["cluster"]
               ))
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, hue_plots, cluster_plots],
            dataset_name="FD001",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD001", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD001", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD001", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(hue_plots, "FD001", use_cache=USE_CACHE["hue"]),
        util.make_dropdown_section(cluster_plots, "FD001", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Klassifizierung",
        "5. Clusteranalyse",
    ]
    eda_panel_FD001 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD001 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD001)

####  EDA-Zusammenfassung FD001
Der Datensatz FD001 enthält konstante Betriebsbedingungen mit einem einzigen Degradationsmodus. Die Triebwerke zeigen eine rechtsschiefe Lebensdauerverteilung mit moderater Streuung.

Die Operation Settings bleiben über den gesamten Lebenszyklus konstant und haben keinen signifikanten Einfluss auf die RUL. Dagegen liefern mehrere Sensoren relevante Informationen: systematische Trends, unterschiedliche Verteilungen über die Lebensdauer hinweg sowie teils signifikante Korrelationen zur RUL.

Die t-SNE/DBSCAN-Clusteranalyse identifiziert zwei Lebenszustände:
- einen stabilen Hauptzustand,
- sowie einen kritischen Frühzustand mit auffälligem Sensorverhalten.

Diese Erkenntnisse bilden die Grundlage für das Feature Engineering und die spätere Modellierung. Eine vollständige Detailauswertung befindet sich in **Anhang B.1**.

#### 3.1.2 Feature Engineering

#### 3.1.3 Modellierung

#### 3.1.4 Hyperparameter-Tuning

#### 3.1.5 Evaluation

---
### 3.2 Analyse für FD002
Der Datensatz FD002 enthält variable Betriebsbedingungen bei gleichzeitig einem einzigen Degradationsmodus. Im Vergleich zu FD001 ist das Verhalten der Sensoren komplexer, da die Betriebspunkte schwanken.
Die Analyse dieses Szenarios liefert wichtige Erkenntnisse darüber, wie stark sich Sensorwerte durch die Operation Settings beeinflussen lassen und welche Normalisierungen oder Features erforderlich sind.

#### 3.2.1 Explorative Datenanalyse (EDA)


In [ ]:
if EDA_FD002:
    # ─────────────────────────────
    # Konfiguration für FD002 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
    "overview": True,
    "ops": True,
    "sensors": True,
    "hue": True,
    "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True

    toggle = util.make_toggle_shortcut(df_train_02, "FD002")
    sensor_cols = [f"sensor_{i}" for i in range(1, 22)]
    df_copy = df_train_02.copy()
    df_copy["op_setting_1_cat"] = pd.qcut(df_copy["op_setting_1"], 3, labels=["tief", "mittel", "hoch"])
    df_copy["op_setting_2_cat"] = pd.qcut(df_copy["op_setting_2"], 3, labels=["tief", "mittel", "hoch"])
    step = (df_copy["op_setting_3"].max() - df_copy["op_setting_3"].min()) / 3
    bins = [df_copy["op_setting_3"].min(), df_copy["op_setting_3"].min()+step, df_copy["op_setting_3"].min()+2*step, df_copy["op_setting_3"].max()]
    df_copy["op_setting_3_cat"] = pd.cut(df_copy["op_setting_3"], bins=bins, labels=["tief","mittel","hoch"], include_lowest=True)

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]
    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[244, 112]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.5, upper_quantile=0.5),
        toggle("2-5. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
        toggle("2-6. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
    ]
    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window = 10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=112, dataset_name="FD002-112"),
        toggle("3-3. Sensor-Korrelation", eda.plot_sensor_correlation_matrix, annot=True),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle),
        toggle("3-5. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation),
        toggle("3-6. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time),
    ]
    hue_plots = [
        toggle("4-1. Sensorverteilungen nach op_setting_1_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=.25, upper_quantile=.75,
                dataset_name="FD002", hue_col="op_setting_1_cat", sensor_cols=sensor_cols)),
        toggle("4-2. Sensorverteilungen nach op_setting_2_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=.25, upper_quantile=.75,
                dataset_name="FD002", hue_col="op_setting_2_cat", sensor_cols=sensor_cols)),
        toggle("4-3. Sensorverteilungen nach op_setting_3_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=.25, upper_quantile=.75,
                dataset_name="FD002", hue_col="op_setting_3_cat", sensor_cols=sensor_cols)),
    ]

    # Einstellungen von plot_tsne_dbscan_clusters
    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_02,
            dataset_name="FD002",
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN,
        )
        df_train_02["cluster_tsne"] = labels

    cluster_plots = [
        toggle("5-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters), # plot_tsne_dbscan_clusters zeigt nur den Plot an
        toggle("5-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("5-3. Zustandsdauer je Cluster (Boxplot)", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("5-4. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("5-5. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD002"),
            fig.update_layout(width=1800, height=600),
            fig
        )),
        toggle("5-6. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("5-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("5-8. Trend Sensoren je Cluster",
            lambda df: util.make_cluster_navigation_panel(
                df=df,
                cluster_col="cluster_tsne",
                cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                sensor_cols=sensor_cols,
                dataset_name="FD002",
                force_recompute=not USE_CACHE["cluster"]
            ))
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, hue_plots, cluster_plots],
            dataset_name="FD002",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD002", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD002", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD002", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(hue_plots, "FD002", use_cache=USE_CACHE["hue"]),
        util.make_dropdown_section(cluster_plots, "FD002", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Klassifizierung",
        "5. Clusteranalyse",
    ]
    eda_panel_FD002 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD002 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD002)

#### EDA-Zusammenfassung FD002  
Der Datensatz FD002 weist variierende Betriebsbedingungen auf, wobei `op_setting_1` und `op_setting_2` stark schwanken und hoch korreliert sind. Die Triebwerke zeigen erneut eine rechtsschiefe Lebensdauerverteilung mit moderater Streuung.  

Die Operation Settings beeinflussen mehrere Sensorverteilungen deutlich, haben aber nur eine geringe direkte Korrelation zur RUL. Die Sensoren sind stark untereinander korreliert, was auf eine hohe Redundanz hinweist. Einzelne Sensoren wie `sensor_14`, `15` und `16` liefern dennoch potenziell prädiktive Informationen.

Die t-SNE/DBSCAN-Clusteranalyse identifiziert sieben Lebenszustände:
- darunter **Cluster 3** als instabiler Zwischenzustand,  
- sowie **Cluster 6** als möglicher finaler Zustand kurz vor Ausfall.

Diese Erkenntnisse bilden die Grundlage für das Feature Engineering und die spätere Modellierung. Eine vollständige Detailauswertung befindet sich in **Anhang B.2**.

#### 3.2.2 Feature Engineering

#### 3.2.3 Modellierung

#### 3.2.4 Hyperparameter-Tuning

#### 3.2.5 Evaluation

---
### 3.3 Analyse für FD003
FD003 weist konstante Betriebsbedingungen, aber mehrere Degradationsmodi auf. Damit eignet sich der Datensatz gut, um rein sensorbasierte Unterschiede im Degradationsverhalten zu untersuchen, ohne dass sich zusätzlich die Betriebszustände verändern.

#### 3.3.1 Explorative Datenanalyse (EDA)

In [ ]:
if EDA_FD003:
    # ─────────────────────────────
    # Konfiguration für FD003 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
        "overview": True,
        "ops": True,
        "sensors": True,
        "hue": True,
        "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True

    toggle = util.make_toggle_shortcut(df_train_03, "FD003")
    sensor_cols = [f"sensor_{i}" for i in [2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 20, 21]]
    df_copy = df_train_03.copy()
    df_copy["op_setting_1_cat"] = pd.qcut(df_copy["op_setting_1"], 3, labels=["tief", "mittel", "hoch"])
    df_copy["op_setting_2_cat"] = pd.qcut(df_copy["op_setting_2"], 3, labels=["tief", "mittel", "hoch"])

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]

    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[99, 55]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.25, upper_quantile=0.75),
        toggle("2-5. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
        toggle("2-6. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
    ]

    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window=10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=55, dataset_name="FD003-55"),
        toggle("3-3. Sensor-Korrelation", lambda df: (
            fig := plt.figure(figsize=(28, 10)),
            axs := fig.subplots(1, 2),
            eda.plot_sensor_correlation_matrix(df, dataset_name="FD003", ax=axs[0]),
            eda.plot_sensor_correlation_matrix(df, sensor_cols=sensor_cols, dataset_name="FD003 (ohne konstante)", annot=True, ax=axs[1]),
            plt.tight_layout(),
        )),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle, sensor_cols=sensor_cols),
        toggle("3-5. Sensorverteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=sensor_cols),
        toggle("3-6. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation, sensor_cols=sensor_cols),
        toggle("3-7. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=sensor_cols),
    ]

    hue_plots = [
        toggle("4-1. Sensorverteilungen nach op_setting_1_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=0.25, upper_quantile=0.75,
                dataset_name="FD003", hue_col="op_setting_1_cat", sensor_cols=sensor_cols)),
        toggle("4-2. Sensorverteilungen nach op_setting_2_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=0.25, upper_quantile=0.75,
                dataset_name="FD003", hue_col="op_setting_2_cat", sensor_cols=sensor_cols)),
    ]

    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_03,
            dataset_name="FD003",
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN,
            dbscan_eps=3,
        )
        df_train_03["cluster_tsne"] = labels

    cluster_plots = [
        toggle("5-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters),
        toggle("5-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("5-3. Zustandsdauer je Cluster (Boxplot)", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("5-4. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("5-5. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD003"),
            fig.update_layout(width=1800, height=600),
            fig)),
        toggle("5-6. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("5-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("5-8. Trend Sensoren je Cluster",
            lambda df: util.make_cluster_navigation_panel(
                df=df,
                cluster_col="cluster_tsne",
                cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                sensor_cols=sensor_cols,
                dataset_name="FD003",
                force_recompute=not USE_CACHE["cluster"]
            )),
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, hue_plots, cluster_plots],
            dataset_name="FD003",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD003", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD003", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD003", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(hue_plots, "FD003", use_cache=USE_CACHE["hue"]),
        util.make_dropdown_section(cluster_plots, "FD003", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Klassifizierung",
        "5. Clusteranalyse",
    ]
    eda_panel_FD003 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD003 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD003)

#### EDA-Zusammenfassung FD003  
Der Datensatz FD003 enthält konstante Betriebspunkte bei mehreren Triebwerktypen und weist eine hohe Streuung in der Lebensdauer auf. Die Lebensdauerverteilung ist erneut rechtsschief, jedoch mit deutlich grösserer Varianz als bei FD001/FD002.

Die Operation Settings bleiben über den gesamten Lebenszyklus konstant und zeigen keinerlei signifikanten Einfluss auf die Sensoren oder die RUL.

Die Sensoren zeigen hingegen ausgeprägte Trends und Strukturen. Mehrere Sensoren korrelieren deutlich mit der Lebensdauer, andere besitzen multimodale Verteilungen oder bilden stark korrelierte Gruppen. Diese Merkmale bieten gute Voraussetzungen für Feature Engineering.

Die Clusteranalyse mit t-SNE/DBSCAN identifiziert fünf Betriebszustände:  
- **Cluster 0** dominiert als stabiler Nominalzustand,  
- **Cluster 1, 2 und 4** zeigen unterschiedliche Degradationsphasen,  
- **Cluster 3** ist kurzzeitig zu beginn aktiv, instabil und wird als Übergangszustand interpretiert.

Diese Struktur liefert eine gute Grundlage für zustandsbasierte Modellierungsansätze. Die vollständige Detailanalyse befindet sich in **Anhang B.3**.


#### 3.3.2 Feature Engineering


#### 3.3.3 Modellierung


#### 3.3.4 Hyperparameter-Tuning


#### 3.3.5 Evaluation

---
### 3.4 Analyse für FD004
FD004 kombiniert die komplexesten Bedingungen aller C-MAPSS-Datensätze: variable Betriebspunkte und mehrere Degradationsmodi.
Dieser Datensatz stellt somit das realistischste, aber auch herausforderndste Szenario dar. Ziel der Analyse ist es, robuste Muster trotz starker Streuung und Rauschen zu identifizieren.


#### 3.4.1 Explorative Datenanalyse (EDA)

In [ ]:
if EDA_FD004:
    # ─────────────────────────────
    # Konfiguration für FD004 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
        "overview": True,
        "ops": True,
        "sensors": True,
        "hue": True,
        "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True

    toggle = util.make_toggle_shortcut(df_train_04, "FD004")
    sensor_cols = [f"sensor_{i}" for i in range(1, 22)]
    df_copy = df_train_04.copy()
    df_copy["op_setting_1_cat"] = pd.qcut(df_copy["op_setting_1"], 3, labels=["tief", "mittel", "hoch"])
    df_copy["op_setting_2_cat"] = pd.qcut(df_copy["op_setting_2"], 3, labels=["tief", "mittel", "hoch"])
    step = (df_copy["op_setting_3"].max() - df_copy["op_setting_3"].min()) / 3
    bins = [df_copy["op_setting_3"].min(), df_copy["op_setting_3"].min() + step, df_copy["op_setting_3"].min() + 2 * step, df_copy["op_setting_3"].max()]
    df_copy["op_setting_3_cat"] = pd.cut(df_copy["op_setting_3"], bins=bins, labels=["tief", "mittel", "hoch"], include_lowest=True)

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]
    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[214, 118]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.25, upper_quantile=0.75),
        toggle("2-5. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
        toggle("2-6. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
    ]
    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window=10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=118, dataset_name="FD004-118"),
        toggle("3-3. Sensor-Korrelation", eda.plot_sensor_correlation_matrix, annot=True),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle, sensor_cols=sensor_cols),
        toggle("3-5. Sensorverteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=sensor_cols),
        toggle("3-6. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation, sensor_cols=sensor_cols),
        toggle("3-7. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=sensor_cols),
    ]
    hue_plots = [
        toggle("4-1. Sensorverteilungen nach op_setting_1_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=.25, upper_quantile=.75,
                dataset_name="FD004", hue_col="op_setting_1_cat", sensor_cols=sensor_cols)),
        toggle("4-2. Sensorverteilungen nach op_setting_2_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=.25, upper_quantile=.75,
                dataset_name="FD004", hue_col="op_setting_2_cat", sensor_cols=sensor_cols)),
        toggle("4-3. Sensorverteilungen nach op_setting_3_cat",
            lambda df: eda.plot_sensor_distributions_by_cycle_range(
                df=df_copy, lower_quantile=.25, upper_quantile=.75,
                dataset_name="FD004", hue_col="op_setting_3_cat", sensor_cols=sensor_cols)),
    ]

    # Einstellungen von plot_tsne_dbscan_clusters
    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_04,
            dataset_name="FD004",
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN,
        )
        df_train_04["cluster_tsne"] = labels

    cluster_plots = [
        toggle("5-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters),
        toggle("5-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("5-3. Zustandsdauer je Cluster (Boxplot)", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("5-4. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("5-5. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD004"),
            fig.update_layout(width=1800, height=600), fig)),
        toggle("5-6. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("5-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("5-8. Trend Sensoren je Cluster",
            lambda df: util.make_cluster_navigation_panel(
                df=df,
                cluster_col="cluster_tsne",
                cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                sensor_cols=sensor_cols,
                dataset_name="FD004",
                force_recompute=not USE_CACHE["cluster"]
            )),
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, hue_plots, cluster_plots],
            dataset_name="FD004",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD004", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD004", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD004", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(hue_plots, "FD004", use_cache=USE_CACHE["hue"]),
        util.make_dropdown_section(cluster_plots, "FD004", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Klassifizierung",
        "5. Clusteranalyse",
    ]
    eda_panel_FD004 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD004 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD004)

#### EDA-Zusammenfassung FD004  
Der Datensatz FD004 kombiniert mehrere Triebwerkstypen mit stark schwankenden Betriebspunkten und langer Lebensdauer. Die Lebensdauerverteilung ist dabei ebenfalls rechtsschief, mit ähnlich grosser Streuung wie in FD003. Die Operation Settings 1 und 2 variieren stark und beeinflussen viele Sensorverteilungen deutlich, während Setting 3 konstant bleibt und vernachlässigt werden kann.

Die Sensorverläufe sind oft verrauscht und zeigen hohe Redundanz, mit wenigen klaren RUL-Trends. Daher ist gezieltes Preprocessing entscheidend.

Die t-SNE/DBSCAN-Clusteranalyse liefert eine gut interpretierbare Struktur mit 18 Zuständen:
- **Cluster 0–9**: Früh- bzw. Zwischenzustände mit teils instabilen Verläufen,  
- **Cluster 10–17**: Klar erkennbare Degradationsphasen am Lebensende.

FD004 stellt hohe Anforderungen an Feature Engineering und Modellierung, bietet aber gleichzeitig Potenzial für robuste, zustandsadaptive Modelle. Eine vollständige Detailauswertung befindet sich in **Anhang B.4**.


#### 3.4.2 Feature Engineering


#### 3.4.3 Modellierung


#### 3.4.4 Hyperparameter-Tuning


#### 3.4.5 Evaluation

---
## 4. Kombinierte Analyse (FD001–FD004)


### 4.1 Explorative Analyse


### 4.2 Feature Engineering


### 4.3 Modellierung


### 4.4 Evaluation

---
## 5. Zusammenfassung und Ausblick


### 5.1 Ergebnisse im Vergleich


### 5.2 Lessons Learned


### 5.3 Weiterführende Ideen

---
---

## Anhang A – Erklärung der verwendeten Plots


Zur besseren Nachvollziehbarkeit der Explorativen Datenanalyse (EDA) werden im Folgenden die wichtigsten Plot-Typen kurz erläutert. Die Beschreibungen beziehen sich nicht auf konkrete Ergebnisse, sondern auf den allgemeinen Zweck und die Lesart der Visualisierungen.

### 1. Lebensdauerkennzahlen  
**Typ:** Tabellenausgabe + Histogramm  
Zentrale Kennzahlen wie Median, Mittelwert, Min/Max und Standardabweichung der Lebensdauer sowie deren Verteilung (z. B. rechtsschief bei Run-to-Failure-Daten).

### 2. Verläufe Operation Settings  
**Typ:** Liniendiagramme einzelner Units  
Visualisiert den zeitlichen Verlauf der Betriebspunkte (`op_setting_1–3`) über den Lebenszyklus. Beurteilt Konstanz oder Variation der Betriebsbedingungen.

### 3. Korrelation zu RUL (Settings/Sensoren)  
**Typ:** Balkendiagramm  
Korrelationskoeffizienten zwischen aktuellen Featurewerten (z. B. im letzten Zyklus) und der Restlebensdauer (RUL). Dient zur Identifikation prädiktiver Merkmale.

### 4. Sensorverläufe einzelner Units  
**Typ:** Liniendiagramme  
Zeigt typische Sensorverläufe über den Lebenszyklus einzelner Triebwerke. Hilft, Trends oder Auffälligkeiten visuell zu erkennen.

### 5. Überlagerte Sensortrends  
**Typ:** Liniendiagramm mit vielen Kurven  
Mehrere Sensorverläufe einer Unit überlagert dargestellt. Dient zur Interpretation von globalen Trends.

### 6. Verteilungen (nach Lebensdauer, Cluster, Settings)  
**Typ:** Box- und Violinplots  
Vergleicht Sensorwerte im letzten Zyklus oder in Lebensdauer-Quantilen – gruppiert nach Cluster oder Betriebspunkt-Kategorie. Zeigt Unterschiede und Streuungen zwischen Gruppen.

### 7. Korrelationsmatrizen  
**Typ:** Heatmaps  
Visualisieren lineare Zusammenhänge zwischen Features (Sensoren/Settings) über den gesamten Datensatz. Unterstützt die Auswahl nicht-redundanter Merkmale.

### 8. Trends über normierte Zeit  
**Typ:** Liniendiagramm (aggregiert)  
Zeigt den mittleren Verlauf eines Features über normierte Zeit (0 = Beginn, 1 = Ausfall). Dient zur Identifikation typischer Degradationstrends.

### 9. t-SNE + DBSCAN  
**Typ:** 2D-Scatterplot  
Dimensionalitätsreduktion und Clustering. Visualisiert unterschiedliche Betriebs- oder Degradationszustände über den gesamten Lebenszyklus hinweg.

### 10. Sankey-Diagramm (Clustertransitions)  
**Typ:** Flussdiagramm  
Zeigt, wie Units im Lebensverlauf von einem Clusterzustand in einen anderen wechseln. Hilft bei der Interpretation typischer Zustandsverläufe.

---
## Anhang B – Explorative Datenanalyse (Detailauswertung)

Dieser Anhang enthält die vollständigen Ergebnisse der explorativen Datenanalyse (EDA) für alle vier C-MAPSS-Teildatensätze. Die Analyse je Datensatz folgt einer einheitlichen Struktur:

- Lebensdaueranalyse  
- Analyse der Operation Settings  
- Sensoranalyse (Trends, Korrelation, Verteilungen)  
- Einfluss der Settings auf Sensorverteilungen  
- Clusteranalyse  
- Gesamtfazit FD00X

**Hinweis:**  
Kurze Zusammenfassungen der wichtigsten Erkenntnisse sind im Hauptteil unter *Kapitel 3* zu finden.  
Die hier dokumentierten Detailanalysen dienen der vollständigen Nachvollziehbarkeit.

### Anhang B.1 – Explorative Detailanalyse FD001

#### 1. Lebensdauer
Die Lebensdauer der Triebwerke in FD001 reicht von 128 bis 362 Zyklen (Median: 199, Mittelwert: 206). Die Verteilung ist rechtsschief – typisch für Run-to-Failure-Daten – mit den meisten Triebwerken im Bereich von 180–230 Zyklen. Die Standardabweichung beträgt ca. 46, was auf eine moderate Streuung hinweist.
#### 2. Operation Settings

- **Plot 2-1:** `op_setting_1` und `op_setting_2` zeigen leichte Schwankungen; `op_setting_3` bleibt konstant bei 100.
- **Plot 2-2:** Keine signifikante Korrelation zwischen den Settings; `op_setting_3` ist konstant.
- **Plot 2-3:** Symmetrische Verteilungen bei `op_setting_1` und `op_setting_2` im letzten Zyklus; `op_setting_3` ohne Variation.
- **Plot 2-4:** Leichte Unterschiede in den Verteilungen nach Lebensdauer (viele vs. wenige Zyklen).
- **Plot 2-5:** Kein klarer Trend über die normierte Zeit.
- **Plot 2-6:** Sehr geringe Korrelation zur RUL (`op_setting_2`: +0.13, `op_setting_1`: –0.05, `op_setting_3`: 0).

**Fazit:** Die Operation Settings sind weitgehend konstant und für die RUL-Vorhersage nur bedingt relevant.
#### 3. Sensoren

- **Plot 3-1:** Sensorverläufe zeigen klare Trends über die Zeit.
  - `sensor_9` und `sensor_14`: stark divergierende Verläufe.
  - `sensor_1`, `sensor_5`, `sensor_10`, `sensor_16`, `sensor_18`, `sensor_19`: konstant → entfernen.

- **Plot 3-2:** Überlagerte Kurven zeigen: einige Sensoren steigen/sinken Richtung Lebensende → potenziell prädiktiv.
- **Plot 3-3:** Korrelationen:
  - `sensor_9` ↔ `sensor_14`: sehr stark (0.96).
  - Cluster:  
    - Positiv korreliert: `sensor_2`, `3`, `4`, `8`, `11`, `13`, `15`, `17`.  
    - Negativ korreliert: `sensor_7`, `12`, `20`, `21` (untereinander positiv).
  - `sensor_6`, `9`, `14`: sonst schwache Korrelationen.
- **Plot 3-4:** Verteilungen im letzten Zyklus:
  - `sensor_9`, `14`: ausgeprägt multimodal.
  - `sensor_2`, `4`, `8`, `17`, `21`: leicht multimodal → Hinweise auf Heterogenität.
- **Plot 3-5:** Verteilungen nach Lebensdauer:
  - Deutliche Unterschiede bei `sensor_8`, `9`, `13`, `14`.
  - Leichte Verschiebungen bei `sensor_2`, `3`, `4`, `7`, `11`, `12`, `15`, `20`, `21`.
- **Plot 3-6:** RUL-Korrelation:
  - `sensor_4`: +0.21 (stärkste positive).
  - `sensor_15`: +0.13 
  - `sensor_12`: –0.19 (stärkste negative).
- **Plot 3-7:** Trends über normierte Zeit:
  - Mehrheit zeigt abnehmende Streuung gegen Lebensende.
  - `sensor_6`: chaotisch/stark schwankend.
  - `sensor_9`, `14`: starke Streuung gegen Ende → nicht stabil.

**Fazit:** Mehrere Sensoren zeigen relevante Trends, Verschiebungen oder Korrelation zur RUL. Konstant bleibende Sensoren werden ausgeschlossen. Clusterstruktur und divergente Sensorverläufe deuten auf unterschiedliche Degradationsmuster hin.
#### 4. Einfluss der Operation Settings auf Sensorverteilungen

- **Plot 4-1:** `op_setting_1_cat` zeigt **kaum Einfluss** auf die Sensorverteilungen. Nur minimale Unterschiede bei `sensor_3`, `sensor_4`, `sensor_7`, `sensor_12`, `sensor_13`.

- **Plot 4-2:** Auch `op_setting_2_cat` verursacht **keine nennenswerten Verschiebungen**. Leichte Unterschiede bei `sensor_7`, `sensor_12`, `sensor_13`, `sensor_15`.

**Fazit:** **Keine relevante Abhängigkeit** der Sensorwerte von den Operation Settings.
#### 5. Clusteranalyse

- **Plot 5-1:** DBSCAN auf t-SNE erkennt zwei Cluster über den gesamten Lebensverlauf → Cluster spiegeln **Zustände der Units** wider.
- **Plot 5-2:** Die Verteilungen der Operation Settings in den Clustern zeigen keine nennenswerten Unterschiede – insbesondere op_setting_3 ist konstant. Die Clusterbildung erfolgt somit unabhängig von den Settings und basiert auf dem Sensorverhalten.
- **Plot 5-3:** Zyklen in Cluster 1 treten **überwiegend früh im Lebenszyklus** auf → Hinweis auf **kritische Betriebsphasen**.
- **Plot 5-4:** Im letzten Zyklus sind alle Units in **Cluster 0** → Cluster 1 tritt **früher im Lebenszyklus** auf.
- **Plot 5-5:** Sankey-Diagramm bestätigt: **kein später Übergang** in Cluster 1 sichtbar.
- **Plot 5-6:** Sensorverteilungen in Cluster 1 weichen stark ab (`sensor_6`, `9`, `13`, `14`).
- **Plot 5-7:** Cluster 1 zeigt **extreme Mittelwerte**, besonders bei `sensor_20`, `12`, `21`.
- **Plot 5-8:**  
  - Cluster 0: stabile, typische Degradationstrends.  
  - Cluster 1: hohe Schwankung → **instabile Betriebsphasen**.

**Fazit:** Cluster erfassen unterschiedliche Lebenszustände. Cluster 1 steht für frühe, potenziell problematische Phasen mit abweichendem Sensorverhalten.


### Gesamtfazit FD001
Die EDA des Datensatzes FD001 zeigt eine moderate Streuung der Lebensdauer mit einer rechtsschiefen Verteilung, wie sie für Run-to-Failure-Prozesse typisch ist. Die Operation Settings bleiben über alle Zyklen konstant und liefern kaum erklärende Information zur RUL.

Im Gegensatz dazu weisen mehrere Sensoren deutliche Trends, unterschiedliche Verteilungen über die Lebensdauer sowie relevante Korrelationen zur RUL auf. Konstant bleibende oder redundante Sensoren wurden identifiziert und ausgeschlossen.

Die t-SNE/DBSCAN-Clusteranalyse ergibt zwei klar unterscheidbare Zustände:

Cluster 0: stabiler Nominalzustand

Cluster 1: früh auftretender, kritischer Zustand mit auffälligem Sensorverhalten

Diese Clusterstruktur bestätigt das Vorliegen unterschiedlicher Degradationsverläufe. Die Kombination aus relevanten Sensoren und identifizierten Zuständen bildet eine solide Grundlage für das anschliessende Feature Engineering und die Modellierung der RUL.



### Anhang B.2 – Explorative Detailanalyse FD002

##### 1. Lebensdauer
Die Lebensdauer der Triebwerke in FD002 reicht von 128 bis 378 Zyklen (Median: 199, Mittelwert: 207). Die Verteilung ist rechtsschief, mit der Mehrheit der Triebwerke im Bereich 180–230 Zyklen. Die Standardabweichung beträgt rund 47 – ähnlich wie bei FD001 – und deutet auf eine moderate Streuung hin.
##### 2. Operation Settings

- **Plot 2-1:** Alle drei Settings (`op_setting_1`, `op_setting_2`, `op_setting_3`) schwanken stark über den Lebensverlauf.
- **Plot 2-2:** `op_setting_1` und `op_setting_2` sind hoch korreliert (+0.94); `op_setting_3` ist nahezu unabhängig.
- **Plot 2-3:** Im letzten Zyklus zeigen `op_setting_1` und `op_setting_2` eine breite Verteilung, `op_setting_3` ist bimodal.
- **Plot 2-4:** Verteilungen unterscheiden sich sehr leicht nach Lebensdauer.
- **Plot 2-5:** Über die normierte Zeit bleibt das Verhalten insgesamt stabil, jedoch mit grosser Streuung.
- **Plot 2-6:** Geringe negative Korrelation zur RUL: `op_setting_2`: –0.12, `op_setting_1`: –0.11, `op_setting_3`: –0.02.

**Fazit:** Die Operation Settings in FD002 sind **variabel und teilweise stark korreliert**, haben aber nur **schwachen Einfluss auf die RUL**.

##### 3. Sensoren

- **Plot 3-1:** Die Sensorverläufe wirken insgesamt **sehr verrauscht**. Einzelne Trends sind visuell schwer erkennbar.
- **Plot 3-2:** Die überlagerten Sensorverläufe wirken insgesamt **sehr verrauscht**. Einzelne Trends sind visuell schwer erkennbar.
- **Plot 3-3:** Die Korrelationen sind **deutlich höher** als in FD001 – viele Sensoren (> 0.9). Nur `sensor_13`, `14`, `20`, `21` zeigen **geringere** und `15` **negative Korrelationen** zu anderen.
- **Plot 3-4:** Im letzten Zyklus zeigen mehrere Sensoren **auffällige Ausreisser** (z. B. `sensor_13`, `14`, `15`).
- **Plot 3-5:** Die RUL-Korrelation ist generell **schwach**.  
  - Positiv: `sensor_16` (+0.123), `sensor_5`, `6`, `7`, `12`, `20`, `21` (alle um +0.1).  
  - Negativ: `sensor_13`, `15`, `19` (aber schwach, max –0.03).
- **Plot 3-6:** Einige Sensoren zeigen **moderate Trends** gegen Ende der normierte Zeit mit teils steigender oder fallender Tendenz (`sensor_11`, `14`, `16`). Die meisten scheinen jedoch fast konstant.

**Fazit:** Trotz hoher Redundanz unter den Sensoren liefern einige Messgrössen (`sensor_11`, `14`, `15`, `16`) potenziell prädiktive Informationen zur RUL. Viele Sensoren sind jedoch stark korreliert oder verhalten sich instabil, was eine gezielte Feature-Selektion notwendig macht.

##### 4. Einfluss der Operation Settings auf Sensorverteilungen

- **Plot 4-1:** `op_setting_1` beeinflusst **mehrere Sensoren sichtbar**, insbesondere `sensor_3`, `4`, `7`, `8`, `9`, `11`, `14`, `15`, `17`.
- **Plot 4-2:** Auch `op_setting_2` zeigt **ähnliche Verschiebungen**, oft bei denselben Sensoren wie bei `op_setting_1`. Erklärbar durch die hohe Korrelation zwischen den beiden Settings.
- **Plot 4-3:** `op_setting_3` beeinflusst nur **wenige Sensoren** deutlich. Die meisten Verteilungen sind klar einseitig zu einer Kategorie konzentriert.

**Fazit:**  
`op_setting_1` und `op_setting_2` haben **signifikanten Einfluss** auf viele Sensorverteilungen. `op_setting_3` hingegen ist meist konstant oder stark einseitig verteilt und trägt **wenig zur Variation** bei.  
→ Beim Feature Engineering muss die **Abhängigkeit einiger Sensoren von den Settings** berücksichtigt werden (z. B. durch Normalisierung nach Settings-Kategorie oder durch Feature-Interaktionen).
##### 5. Clusteranalyse

- **Plot 5-1:** DBSCAN auf t-SNE identifiziert **7 Cluster** → deutliche Trennung potenzieller Zustände.
- **Plot 5-2:** Die Verteilungen der Operation Settings unterscheiden sich deutlich zwischen den Clustern, besonders bei `op_setting_1` und `op_setting_2`. Dies weist darauf hin, dass die Clusterbildung teilweise durch unterschiedliche Betriebsbedingungen beeinflusst wird.
- **Plot 5-3:** **Cluster 3** weist die **niedrigste Lebensdauer** auf → möglicher **instabiler Zustand**.
- **Plot 5-4:** Im letzten Zyklus dominieren **Cluster 1, 4, 5** → stabilere Betriebszustände.
- **Plot 5-5:** Sankey zeigt:  
  - **Cluster 6** tritt **erst sehr spät** auf → möglicher **finaler Zustand kurz vor Ausfall**.  
  - **Cluster 3** verschwindet über die Zeit vollständig, endet aber in keiner Unit → instabiler, temporärer Zustand.
- **Plot 5-6:** Sensorverteilungen unterscheiden sich stark, v.a. in `sensor_3`, `4`, `9`, `11`, `15`, `17`, `20`, `21`.
- **Plot 5-7:** Mittlere Sensorwerte:
  - **Cluster 2:** zeigt **viele extreme Abweichungen** → stark vom Rest abweichender Zustand.
  - **Cluster 4/5:** homogenes, stabiles Sensorbild → möglicher **Normalbetrieb**.
  - **Cluster 6:** einzelne starke Abweichungen → **späte Phase**.
- **Plot 5-8:** Zeitliche Sensorverläufe pro Cluster:
  - **Cluster 0, 1, 4, 5:** stabile und gut interpretierbare Trends → typisch für regelmässige Betriebsphasen.
  - **Cluster 2:** untypische, stark streuende Verläufe → deutliche Abweichung vom Normalverhalten.
  - **Cluster 3:** schwankend, verschwindet im Verlauf → instabiler Zwischenzustand.
  - **Cluster 6:** stark unruhig, v.a. gegen Ende → vermutlich durch **kurze Verweildauer** bedingt (wenig Glättung möglich).

**Fazit:** Die 7 Cluster bilden unterschiedliche Betriebs- und Degradationszustände ab.  
**Cluster 3** deutet auf einen kritischen Zwischenzustand hin, **Cluster 6** auf den finalen Zustand kurz vor Ausfall.  
Starke Abweichungen in den Sensorverteilungen und Mittelwerten – insbesondere in den Clustern 2, 3 und 6 – können als **Frühwarnindikatoren für kritische Phasen** dienen.

### Gesamtfazit

Die EDA für FD002 zeigt ein komplexeres Verhalten als FD001:

- Die **Lebensdauerverteilung** ist erneut rechtsschief mit ähnlicher Streuung, aber mehr Units (260).
- Die **Operation Settings** schwanken deutlich und beeinflussen mehrere Sensoren stark (v. a. `op_setting_1`, `2`). Trotzdem bleibt ihr **direkter Einfluss auf die RUL gering**.
- Die **Sensoren** zeigen hohe Redundanz mit vielen starken Korrelationen. Einzelne Sensoren (z. B. `sensor_14`, `15`, `16`) liefern dennoch **potenziell prädiktive Informationen** – aber nur schwach.
- Die **Sensorverteilungen hängen klar von den Settings** ab → beim Feature Engineering muss die Abhängigkeit berücksichtigt werden.
- Die **Clusteranalyse** offenbart deutliche Zustandswechsel über den Lebenszyklus:
  - **Cluster 3** verschwindet früh und endet nie → **instabiler Zwischenzustand**.
  - **Cluster 6** tritt kurz vor Ausfall auf → **finaler Zustand**.
  - **Cluster 2** zeigt extreme Sensorabweichungen.
  - **Cluster 4 und 5** stehen für stabilen Betrieb.

Insgesamt liefert die EDA wichtige Hinweise für das Feature Engineering:
- Redundante Sensoren können reduziert werden.
- Betriebszustände (Cluster) ermöglichen eine differenzierte Betrachtung der Degradation.
- Frühzeitiges Erkennen von kritischen Clustern könnte die Modellgüte erhöhen.

### Anhang B.3 – Explorative Detailanalyse FD003

##### 1. Lebensdauer
Die Lebensdauer der Triebwerke in FD003 reicht von 140 bis 525 Zyklen (Median: 228, Mittelwert: 241). Die Verteilung ist rechtsschief, mit der Mehrheit der Triebwerke im Bereich 180–270 Zyklen. Die Standardabweichung beträgt rund 73 – deutlich höher als bei FD001/FD002 – und deutet auf eine grosse Streuung hin.
##### 2. Operation Settings  
- **Plot 2-1:** Alle drei Settings (`op_setting_1`, `op_setting_2`, `op_setting_3`) sind **nahezu konstant** über den Lebensverlauf.  
- **Plot 2-2:** `op_setting_1` und `op_setting_2` sind **nicht korreliert** (r ≈ 0), `op_setting_3` ist **komplett konstant**.  
- **Plot 2-3:** Im letzten Zyklus zeigen `op_setting_1` und `op_setting_2` eine **sehr schmale Verteilung**, `op_setting_3` ist **identisch für alle**.  
- **Plot 2-4:** Auch über die Lebensdauerklassen hinweg kaum Variation → **kein Einfluss der RUL sichtbar**.  
- **Plot 2-5:** Verlauf über normierte Zeit bleibt **stabil mit geringer Streuung**.  
- **Plot 2-6:** Geringe negative Korrelation zur RUL: `op_setting_2`: –0.13, `op_setting_1`: –0.05, `op_setting_3`: 0.00.

**Fazit:** In FD003 sind die Operation Settings weitgehend **konstant** und zeigen **keinen relevanten Einfluss auf die Lebensdauer**. `op_setting_3` kann vollständig ignoriert werden.
##### 3. Sensoren

- **Plot 3-1:** Die Sensorverläufe einzelner Units zeigen **teils deutliche Trends**, laufen jedoch oft auseinander.
- **Plot 3-2:** Der überlagerte Verlauf (z. B. Unit 64) zeigt **ausgeprägte Strukturen** – mehrere Sensoren steigen oder fallen deutlich gegen Ende an.
- **Plot 3-3:** Die Korrelationen sind **hoch**, besonders unter `sensor_2`, `3`, `4`, `8`, `9`, `11`, `13`, `17` und unter  `7`, `12`, `20`, `21`.
  `sensor_6`, `15` zeigen hingegen **negative oder schwache Korrelationen**.
- **Plot 3-4:** Im letzten Zyklus zeigen sich oft **biomodale** Verteilungen.
- **Plot 3-5:** Die Lebensdauer beeinflusst alle Sensoren sichtbar.
- **Plot 3-6:** Stärkste RUL-Korrelationen haben `sensor_11`, `8`, `15`, `2` (alle positiv), sowie `sensor_20` (negativ, –0.13).
- **Plot 3-7:** Über die normierte Zeit zeigen viele Sensoren **regelmässige Trends**, v.a. `sensor_2`, `3`, `4`, `8`, `9`, `11`, `13`, `17`.

**Fazit:**  
FD003 weist **viele stabile und regelmässige Sensorverläufe** auf, besonders bei `sensor_2`, `3`, `4`, `8`, `9`, `11`, `13`, `17`.  
Einige Sensoren zeigen **starke Korrelation zur Lebensdauer**, v.a. `sensor_11` und `20`.  
Auffällig sind **biomodale Verteilungen** im letzten Zyklus sowie **zwei Gruppen starker Korrelationen** unter den Sensoren.  
→ Diese Merkmale können gezielt für das Feature Engineering und die Zustandsklassifikation genutzt werden.
##### 4. Einfluss der Operation Settings auf Sensorverteilungen

- **Plot 4-1:** `op_setting_1` beeinflusst viele Sensoren leicht.
- **Plot 4-2:** Auch `op_setting_2` zeigt teils ähnliche Verschiebungen.

**Fazit:**  
Die Operation Settings in FD003 zeigen **leichte Einflüsse auf einzelne Sensoren**, sind aber **weniger stark** als in FD002.  
##### 5. Clusteranalyse

- **Plot 5-1:** DBSCAN auf t-SNE identifiziert **5 Cluster** (0–4) und einige **Rauschenpunkte** (–1). Cluster 0 dominiert deutlich.
- **Plot 5-2:** Die Verteilungen der Operation Settings unterscheiden sich nicht signifikant zwischen den Clustern. `op_setting_1` und `op_setting_2` zeigen leichte Schwankungen, `op_setting_3` bleibt konstant bei 100. → Clusterbildung basiert primär auf den Sensorwerten, nicht auf Betriebsbedingungen.
- **Plot 5-3:** **Cluster 3** tritt überwiegend **früh** auf, **Cluster 4** fast nur **spät** → Übergangs- bzw. Endzustände. **Cluster 0** erscheint über die gesamte Lebensdauer hinweg → typischer Betriebszustand.
- **Plot 5-4:** Im letzten Zyklus sind vor allem **Cluster 0 und 2** vertreten → mögliche Normalzustände.
- **Plot 5-5:** Sankey zeigt:  
  - **Cluster 0** bleibt über fast die gesamte Lebensdauer aktiv, verzweigt spät in **Cluster 1, 4**.  
  - **Cluster 3** ist **kurzlebig** und **instabil**, verschwindet früh.
- **Plot 5-6:** Sensorverteilungen unterscheiden sich leicht je Cluster.
- **Plot 5-7:** Mittlere Sensorwerte:  
  - **Cluster 1/2/4:** hohe Werte in mehreren Sensoren → möglicher **Endzustand**.  
  - **Cluster 0:** zentrales Cluster mit Werten nahe dem Mittel.  
  - **Cluster 3:** teils starke negative Abweichungen.
- **Plot 5-8:** Die normierten Sensortrends zeigen cluster-spezifische Muster:  
  - Cluster 0: Stabile Verläufe mit spätem Anstieg vieler Sensoren.  
  - Cluster 1: Klarer **End-of-Life Trend** in fast allen Sensoren.  
  - Cluster 2: Kurze Phase am Lebensende mit starkem Anstieg.  
  - Cluster 3: Keine klaren Muster, hohe Streuung – evtl. **Rauschen**.  
  - Cluster 4: Deutliche **Degradationsphase**, die später beginnt, aber viele Sensoren betrifft.

**Fazit:**  
Die Cluster lassen sich als **verschiedene Zustände im Lebenszyklus der Triebwerke** interpretieren:  
- **Cluster 0** stellt den **nominalen Betriebszustand** dar, der lange anhält.  
- **Cluster 1, 2, 4** markieren **verschiedene Degradationsphasen** gegen Ende der Lebensdauer.  
- **Cluster 3** ist kurzzeitig aktiv, unregelmässig und wahrscheinlich ein **artefaktbelasteter Übergangszustand**.  

Die Cluster korrelieren sowohl mit der Zeit als auch mit charakteristischen **Sensorverläufen**, was eine **zustandsbasierte Modellierung und Prognose** der Restlebensdauer ermöglicht.

### Gesamtfazit

Der FD003-Datensatz ist geprägt durch **lange Lebensdauer mit hoher Streuung**, **konstante Betriebspunkte** und **deutliche sensorbasierte Degradationsmuster**.  

Die **Operation Settings** sind über die gesamte Zeit **nahezu konstant** und haben **keinen relevanten Einfluss auf die Sensoren oder Lebensdauer**.  

Die **Sensorverläufe** zeigen hingegen klare Strukturen:  
- Mehrere Sensoren besitzen **stabile, RUL-abhängige Trends**.  
- mit teils **starker Korrelation zur Lebensdauer**. 
- Auffällig sind **bi- bzw. multimodale Verteilungen** im letzten Zyklus.

Die **Clusteranalyse** ergibt ein **valides Zustandsmodell**:  
- **Cluster 0** als **Nominalzustand** über lange Zeit,  
- **Cluster 1, 2, 4** als **Degradationsphasen**,  
- **Cluster 3** ist instabil und wahrscheinlich ein **Übergangszustand**.

→ FD003 eignet sich besonders gut für **zustandsbasierte Modelle** und **Feature Engineering auf Basis stabiler Sensortrends**.

### Anhang B.4 – Explorative Detailanalyse FD004

##### 1. Lebensdauer
Die Lebensdauer der Triebwerke in FD004 reicht von 128 bis 543 Zyklen (Median: 234, Mittelwert: 246). Die Verteilung ist rechtsschief, mit der Mehrheit der Triebwerke im Bereich 190–290 Zyklen. Die Standardabweichung beträgt rund 73 – ähnlich wie bei FD003 – und deutet auf eine grosse Streuung hin.
##### 2. Operation Settings
- **Plot 2-1:** Alle drei Settings (`op_setting_1`, `op_setting_2`, `op_setting_3`) zeigen **keine konstanten Verläufe**, sondern **starke Schwankungen über die Zeit**.
- **Plot 2-2:** `op_setting_1` und `op_setting_2` sind **hoch korreliert** (r = 0.94), `op_setting_3` ist **unkorreliert**.
- **Plot 2-3:** Im letzten Zyklus weisen `op_setting_1` und `op_setting_2` eine **breite Verteilung** auf, `op_setting_3` ist dagegen **fast konstant**.
- **Plot 2-4:** Keine systematische Abhängigkeit der Werte von der Lebensdauerklasse sichtbar → **kein direkter Einfluss auf RUL**.
- **Plot 2-5:** Über normierte Zeit bleibt der **Verlauf stabil**.
- **Plot 2-6:** **Sehr schwache Korrelationen** mit der Lebensdauer:  
  - `op_setting_1`: –0.02  
  - `op_setting_2`: +0.004  
  - `op_setting_3`: –0.033

**Fazit:**  
In FD004 schwanken die Operation Settings stark, beeinflussen die Lebensdauer aber **nicht signifikant**. `op_setting_3` ist **weitgehend konstant**.
##### 3. Sensoren  

- **Plot 3-1:** Die Sensorverläufe wirken insgesamt **sehr verrauscht**. Einzelne Trends sind visuell schwer erkennbar.
- **Plot 3-2:** Sensorverläufe sind **chaotisch und überlagert**, kaum erkennbare Trends.  
- **Plot 3-3:** Es bestehen **sehr starke Korrelationen** zwischen fast allen Sensoren (Ausnahmen: `sensor_8`, `13`, `15`, `19`).  
  → **Starke Redundanz**, viele Sensoren sind fast identisch.  
  `sensor_15` hat viele **negative Korrelationen**.  
- **Plot 3-4:** Im letzten Zyklus haben viele Sensoren eine **ähnliche multimodale Verteilungen**.  
- **Plot 3-5:** RUL-Korrelationen sind **sehr schwach** (keine über ±0.03).  
- **Plot 3-6:** Auch über die normierte Zeit zeigen sich **kaum klare Trends**.

**Fazit:**  
FD004 ist stark durch **Sensorredundanz und verrauschte Verläufe** geprägt.  
Es gibt **keine klaren RUL-Trends**, die Sensoren sind **wenig aussagekräftig**.  
→ **Feature Engineering muss auf Dimensionalitätsreduktion und Rauschunterdrückung fokussieren.**
##### 4. Einfluss der Operation Settings auf Sensorverteilungen  

- **Plot 4-1 bis 4-3:** `op_setting_1` und `op_setting_2` zeigen bei vielen Sensoren **deutliche Verschiebungen** der Verteilungen.  
- `op_setting_3` hingegen ist **fast konstant** – nur minimale Unterschiede sichtbar.  
- Die Wirkung ist bei **`op_setting_1` und `2` unterschiedlich stark je Sensor**, oft multimodal.  

**Fazit:**  
In FD004 haben **`op_setting_1` und `op_setting_2` einen klaren Einfluss auf viele Sensoren**.  
Diese Effekte müssen beim **Feature Engineering berücksichtigt und ggf. kompensiert werden**.  
`op_setting_3` kann hingegen **vermutlich ignoriert** werden.
##### 5. Clusteranalyse

- **Plot 5-1:** DBSCAN auf t-SNE identifiziert **18 Cluster** (0–17) und etwas **Rauschen** (–1). Die Cluster wirken **kompakt und gut trennbar**.
- **Plot 5-2:** Die Operation Settings zeigen deutliche Unterschiede zwischen den Clustern:
  - `op_setting_1` und `op_setting_2` variieren stark über die Cluster hinweg und bilden klar abgegrenzte Betriebsgruppen.
  - `op_setting_3` ist meist konstant bei 100, jedoch zeigen Cluster 3 und 6 Werte um 60 → Hinweis auf abweichende Steuerungsmodi.
- **Plot 5-3:** Die Lebensdauerverteilung pro Cluster ist **sehr unterschiedlich**.  
  - Cluster 0–8: **frühe Zyklen** → mögliche **Startzustände**.  
  - Cluster 9–17: **späte Zyklen**, mittlere bis hohe Restlebensdauer → **Endzustände**.
- **Plot 5-4:** Im letzten Zyklus sind v.a. Cluster **2** und **16** dominant → typische **Endzustände**.
- **Plot 5-5:** Sankey zeigt viele **komplexe Übergänge**, besonders:  
  - **Cluster 2** bleibt **lange stabil**.  
  - Übergänge zu Clustern **13–17** treten meist **erst spät** auf.
- **Plot 5-6:** Sensorverteilungen variieren klar zwischen Clustern wobei sich manche teils überlagern.
- **Plot 5-7:** Die mittleren Sensorwerte zeigen **charakteristische Muster je Cluster**.  
  - Einige Cluster weisen **starke Abweichungen in vielen Sensoren** auf.  
  - Cluster 3, 6 haben z. T. **negative Mittelwerte** in vielen Sensoren während 5, 11 viele **positive Mittelwerte** haben.
  - **Plot 5-8:** Normierte Sensorverläufe pro Cluster zeigen cluster-spezifische Muster:  
    - **Cluster 0–9:** Stabile Trends oder verrauschte Verläufe über gesamte Lebensdauer.  
    - **Cluster 10–17:** **Treten erst gegen Ende auf**, zeigen dann **klare Degradationsmuster** (stark steigende Werte, hohe Varianz).  

**Fazit:**  
Die Cluster in FD004 lassen sich gut voneinander abgrenzen und bilden verschiedene Zustände im Lebenszyklus ab.
Sie spiegeln dabei nicht nur zeitliche Degradationsphasen, sondern auch klar unterscheidbare Betriebsmodi (Operation Settings) wider.
→ Das Clustering bietet eine valide Zustandsklassifikation, die für Feature Engineering und zustandsbasierte Prognosemodelle genutzt werden kann.

### Gesamtfazit

Der FD004-Datensatz ist durch **lange Lebensdauern** mit **hoher Streuung**, **stark schwankende Betriebspunkte**, **verrauschte Sensorverläufe** und **starke Redundanz** in den Messwerten geprägt.  

Die **Operation Settings 1 & 2 beeinflussen viele Sensoren deutlich**, müssen also beim Feature Engineering berücksichtigt werden. **Setting 3 ist weitgehend konstant** und kann ignoriert werden.  

Trotz der chaotischen Einzelverläufe zeigt die **Clusteranalyse mit t-SNE & DBSCAN** klare Zustandsmuster:  
- **Cluster 0–9**: Frühphasen oder Startzustände – viele verrauschte oder unklare Verläufe.  
- **Cluster 10–17**: Spätphasen – klar erkennbare Degradationstrends mit hoher Aussagekraft.  

→ Für Prognosemodelle empfiehlt sich eine **clusterbasierte Segmentierung** oder gezielte **Feature-Reduktion & Rauschunterdrückung**, um aussagekräftige Muster zu extrahieren.  
Die **Cluster korrelieren klar mit den Betriebspunkten** (Operation Settings), was auf **systematische Zustände im Lebenszyklus** hinweist.  
FD004 eignet sich gut für die Entwicklung **zustandsadaptiver Modelle**, erfordert jedoch ein **aufwändiges Preprocessing** aufgrund der **starken Heterogenität** und **hohen Sensor-Redundanz**.